In [1]:
import docplex
import pandas as pd
import tensorflow as tf
import numpy as np
import utility
import copy
import mlp_explainer
import mymetrics
import time
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from milp import codify_network
from teste import get_minimal_explanation
from sklearn.metrics import classification_report
import pickle

In [2]:
#SklearnDatasets
dataset = datasets.load_wine()
df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
scaler = MinMaxScaler()
scaler.fit(dataset.data)
scaled_df = scaler.transform(dataset.data)
lower_bound = scaled_df.min()
upper_bound = scaled_df.max()
print(lower_bound, upper_bound)
df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
targets = dataset.target
df_scaled['target'] = targets
columns = df_scaled.columns
dataset_name = 'Wine'
result_path = f'{dataset_name}_results'
if not os.path.exists(result_path):
    os.makedirs(result_path)
    print(f"Created directory: {result_path}")
else:
    print(f"Directory already exists: {result_path}")
display(df_scaled)

0.0 1.0
Directory already exists: Wine_results


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,0.842105,0.191700,0.572193,0.257732,0.619565,0.627586,0.573840,0.283019,0.593060,0.372014,0.455285,0.970696,0.561341,0
1,0.571053,0.205534,0.417112,0.030928,0.326087,0.575862,0.510549,0.245283,0.274448,0.264505,0.463415,0.780220,0.550642,0
2,0.560526,0.320158,0.700535,0.412371,0.336957,0.627586,0.611814,0.320755,0.757098,0.375427,0.447154,0.695971,0.646933,0
3,0.878947,0.239130,0.609626,0.319588,0.467391,0.989655,0.664557,0.207547,0.558360,0.556314,0.308943,0.798535,0.857347,0
4,0.581579,0.365613,0.807487,0.536082,0.521739,0.627586,0.495781,0.490566,0.444795,0.259386,0.455285,0.608059,0.325963,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,0.705263,0.970356,0.582888,0.510309,0.271739,0.241379,0.056962,0.735849,0.205047,0.547782,0.130081,0.172161,0.329529,2
174,0.623684,0.626482,0.598930,0.639175,0.347826,0.282759,0.086498,0.566038,0.315457,0.513652,0.178862,0.106227,0.336662,2
175,0.589474,0.699605,0.481283,0.484536,0.543478,0.210345,0.073840,0.566038,0.296530,0.761092,0.089431,0.106227,0.397290,2
176,0.563158,0.365613,0.540107,0.484536,0.543478,0.231034,0.071730,0.754717,0.331230,0.684300,0.097561,0.128205,0.400856,2


In [3]:
np.random.seed(50)
X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
X = np.concatenate((X_train,X_test),axis=0)
y = np.concatenate((y_train,y_test),axis=0)

training_data = pd.DataFrame(X_train, columns = columns[:-1])
training_data[columns[-1]] = y_train
testing_data = pd.DataFrame(X_test, columns = columns[:-1])
testing_data[columns[-1]] = y_test
dataframe = pd.concat([training_data, testing_data])
data = dataframe.to_numpy()
n_classes = dataframe['target'].nunique()

original_bounds = [[dataframe[dataframe.columns[i]].min(),dataframe[dataframe.columns[i]].max()] for i in range(len(dataframe.columns[:-1]))]
keras_model = tf.keras.models.load_model(f'new_models/{dataset_name}.h5')

In [4]:
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)
# with open(f'{result_path}/{dataset_name}_mp_model.pkl', 'wb') as f:
#     pickle.dump(mp_model, f)

# with open(f'bounds/{dataset_name}_output_bounds.pkl', 'wb') as f:
#     pickle.dump(output_bounds, f)

In [5]:
predictions = []
possible_classes = np.unique(y_test)
class_indexes = []
class_predictions = []
for i in range(n_classes):
    class_indexes.append([])
    class_predictions.append([])
possible_classes, class_indexes, class_predictions
data = testing_data.to_numpy()
for i in range(len(data)):
    predictions.append(mlp_explainer.model_classification_output(k_model=keras_model, net_input=data[i, :-1])[1].numpy())    
    for j,p_class in enumerate(possible_classes):
        if predictions[-1] == p_class:
            class_indexes[j].append(i)
            class_predictions[j].append(data[i, :-1])
print("Metrics:", classification_report(testing_data.to_numpy()[:, -1], predictions,digits=4))

Metrics:               precision    recall  f1-score   support

         0.0     0.9767    0.9545    0.9655        44
         1.0     0.9623    0.9444    0.9533        54
         2.0     0.9474    1.0000    0.9730        36

    accuracy                         0.9627       134
   macro avg     0.9621    0.9663    0.9639       134
weighted avg     0.9630    0.9627    0.9626       134



In [6]:
cols = list(testing_data.columns)
if 'target' not in cols:
    cols.append('target')
predicted_dataset = []
for i,pos_class in enumerate(np.unique(y_test)):
    for instance in (testing_data.to_numpy()[:, :-1][class_indexes[i]]):
        instance = np.append(instance, pos_class.astype('int'))
        predicted_dataset.append(instance)
predicted_dataset = np.asarray(predicted_dataset)
pred_dataset_df = pd.DataFrame(predicted_dataset, columns=cols)
pred_dataset_df['target'] = pred_dataset_df['target'].astype('int')

In [7]:
# Save Bounds to CSV
#np.savez(f'bounds/{dataset_name}_data_bounds.npz', original_bounds=original_bounds)
# Save Testing Set to CSV
#pred_dataset_df.to_csv(f'{dataset_name}_results/{dataset_name}_X_test.csv', index=False)

In [16]:
metrics_dataframes = []
times_onestep = []
sizes_onestep = []
rsum_onestep = []
coverage_onestep = []
pos_exp_onestep = []
neg_exp_onestep = []
onestep_explanations = []

In [17]:
original_bounds

[[0.0, 1.0],
 [0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 0.9999999999999998],
 [0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0]]

In [18]:
def compute_mean_std(arr):
    return np.mean(arr), np.std(arr)

def relative_percentage_diff(new, old):
    if np.any(old == 0):
        print(f'Warning: found possible division by zero')
        return np.where(old != 0, ((new - old) / old) * 100, np.nan)
    return ((new - old) / old) * 100

p_value = 0.25
print(f"p = {p_value}")
times_twostep = []
sizes_twostep = []
rsum_twostep = []
coverage_twostep = []
twostep_explanations = []
for j in range(len(pred_dataset_df['target'].unique())):
    for i, sample in enumerate((testing_data.to_numpy()[:, :-1][class_indexes[j]])):
        start = time.perf_counter()
        print(f'Explaining: {sample}')
        
        explanation, minimal = mlp_explainer.run_explanation_doublestep(sample = sample, n_classes=n_classes, kmodel=keras_model, model=mp_model, output_bounds=output_bounds, og_bounds=original_bounds, p=p_value)
        end = time.perf_counter()
        twostep_explanations.append(explanation)
        times_twostep.append(end-start)
        sizes_twostep.append(len(minimal))
        rsum_twostep.append(mymetrics.range_sum(twostep_explanations[-1]))
        
        coverage_twostep.append(len(mymetrics.calculate_coverage(testing_data, twostep_explanations[-1])))
        print(f'explanation {explanation}\n')

        start = time.perf_counter()
        explanation, minimal = mlp_explainer.run_explanation(sample = sample, n_classes=n_classes, kmodel=keras_model, model=mp_model, output_bounds=output_bounds, og_bounds=original_bounds, enable_log=False,
                                                             )
        end = time.perf_counter()
        onestep_explanations.append(explanation)
        times_onestep.append(end-start)
        sizes_onestep.append(len(minimal))
        rsum_onestep.append(mymetrics.range_sum(onestep_explanations[-1]))
        coverage_onestep.append(len(mymetrics.calculate_coverage(testing_data, onestep_explanations[-1])))
        
times_onestep = np.array(times_onestep)
times_twostep = np.array(times_twostep)
sizes_onestep = np.array(sizes_onestep)
sizes_twostep = np.array(sizes_twostep)
rsum_onestep = np.array(rsum_onestep)
rsum_twostep = np.array(rsum_twostep)
coverage_onestep = np.array(coverage_onestep)
coverage_twostep = np.array(coverage_twostep)

# Compute means and standard deviations
(time_mean_onestep, time_std_onestep) = compute_mean_std(times_onestep)
(time_mean_twostep, time_std_twostep) = compute_mean_std(times_twostep)

(sizes_mean_onestep, sizes_std_onestep) = compute_mean_std(sizes_onestep)
(sizes_mean_twostep, sizes_std_twostep) = compute_mean_std(sizes_twostep)

(rsum_mean_onestep, rsum_std_onestep) = compute_mean_std(rsum_onestep)
(rsum_mean_twostep, rsum_std_twostep) = compute_mean_std(rsum_twostep)

(coverage_mean_onestep, coverage_std_onestep) = compute_mean_std(coverage_onestep)
(coverage_mean_twostep, coverage_std_twostep) = compute_mean_std(coverage_twostep)

# Compute relative percentage differences (Mean & Std)
time_mean_diff = relative_percentage_diff(time_mean_twostep, time_mean_onestep)
sizes_mean_diff = relative_percentage_diff(sizes_mean_twostep, sizes_mean_onestep)
rsum_mean_diff = relative_percentage_diff(rsum_mean_twostep, rsum_mean_onestep)
coverage_mean_diff = relative_percentage_diff(coverage_mean_twostep, coverage_mean_onestep)

time_std_diff = relative_percentage_diff(time_std_twostep, time_std_onestep)
sizes_std_diff = relative_percentage_diff(sizes_std_twostep, sizes_std_onestep)
rsum_std_diff = relative_percentage_diff(rsum_std_twostep, rsum_std_onestep)
coverage_std_diff = relative_percentage_diff(coverage_std_twostep, coverage_std_onestep)

# Compute pointwise relative differences
time_relative_pointwise = relative_percentage_diff(times_twostep, times_onestep)
sizes_relative_pointwise = relative_percentage_diff(sizes_twostep, sizes_onestep)
rsum_relative_pointwise = relative_percentage_diff(rsum_twostep, rsum_onestep)
coverage_relative_pointwise = relative_percentage_diff(coverage_twostep, coverage_onestep)

# Compute pointwise means
time_relative_mean = np.mean(time_relative_pointwise) 
sizes_relative_mean = np.mean(sizes_relative_pointwise)
rsum_relative_mean = np.mean(rsum_relative_pointwise)
coverage_relative_mean = np.mean(coverage_relative_pointwise)

# Compute pointwise standard deviations
time_relative_std = np.std(time_relative_pointwise) 
sizes_relative_std = np.std(sizes_relative_pointwise)
rsum_relative_std = np.std(rsum_relative_pointwise)
coverage_relative_std = np.std(coverage_relative_pointwise)

# Organize Data
all_metrics_data = {
    'Metric': ['Time', 'Size', 'Ranges_Sum', 'Coverage'],
    'ONESTEP_MEAN': [time_mean_onestep, sizes_mean_onestep, rsum_mean_onestep, coverage_mean_onestep],
    'ONESTEP_STD': [time_std_onestep, sizes_std_onestep, rsum_std_onestep, coverage_std_onestep],
    'TWOSTEP_MEAN': [time_mean_twostep, sizes_mean_twostep, rsum_mean_twostep, coverage_mean_twostep],
    'TWOSTEP_STD': [time_std_twostep, sizes_std_twostep, rsum_std_twostep, coverage_std_twostep],
    'MEAN_DIFF_%': [time_mean_diff, sizes_mean_diff, rsum_mean_diff, coverage_mean_diff],
    'STD_DIFF_%': [time_std_diff, sizes_std_diff, rsum_std_diff, coverage_std_diff],
    'POINTWISE_MEAN_%': [time_relative_mean, sizes_relative_mean, rsum_relative_mean, coverage_relative_mean],
    'POINTWISE_STD_%': [time_relative_std, sizes_relative_std, rsum_relative_std, coverage_relative_std]
}
# Display and save
all_metrics_df = pd.DataFrame(all_metrics_data)
display(all_metrics_df)
#all_metrics_df.to_csv(f'{result_path}/results_{p_value}.csv', index=False)

#Save Raw Metric Data
raw_df = pd.DataFrame({
    "times_onestep": times_onestep, 
    "times_twostep": times_twostep,
    "sizes_onestep": sizes_onestep, 
    "sizes_twostep": sizes_twostep,
    "rsum_onestep": rsum_onestep, 
    "rsum_twostep": rsum_twostep,
    "coverage_onestep": coverage_onestep, 
    "coverage_twostep": coverage_twostep,
    "time_relative_%": time_relative_pointwise,
    "sizes_relative_%": sizes_relative_pointwise,
    "rsum_relative_%": rsum_relative_pointwise,
    "coverage_relative_%": coverage_relative_pointwise
})

# # Save to CSV
# raw_df.to_csv(f"{result_path}/raw_metric_data_{p_value}.csv", index=False)

# # Save onestep explanations
# np.savez(f'{result_path}/onestep_explanations_{p_value}.npz', 
#          onestep_explanations=onestep_explanations)

# # Save twostep explanations
# np.savez(f'{result_path}/twostep_explanations{p_value}.npz', 
#          twostep_explanations=twostep_explanations)

p = 0.25
Explaining: [0.78684211 0.18577075 0.45454545 0.27835052 0.2826087  0.57586207
 0.41983122 0.24528302 0.49526814 0.29180887 0.45528455 0.84981685
 0.53994294]
explanation [[0.0, 1.0], [0.0, 1.0], [0.0, 0.9999999999999999], [0.0, 0.32277484958149477], [0.2419976240290483, 0.47547027395138164], [0.520249080372515, 1.0], [0.3260625046592572, 0.4673268000500339], [0.0, 0.3104285849134652], [0.0, 0.5282428871158177], [0.280311343970082, 0.3800818294478302], [0.40806940771215794, 0.46727389937172414], [0.8415693139542668, 0.8756703224741414], [0.5375303961375624, 1.0]]

Explaining: [0.71052632 0.15019763 0.71657754 0.61340206 0.33695652 0.69655172
 0.61392405 0.30188679 0.6214511  0.37713311 0.57723577 0.52747253
 0.71825963]
explanation [[0.6720596081404459, 1.0], [0.0, 1.0], [0.0, 0.9999999999999999], [0.0, 0.6270818675427696], [0.0, 0.35410249693923607], [0.6878538406979738, 1.0], [0.0, 0.9999999999999998], [0.0, 0.31551745039307955], [0.0, 0.6411436958346333], [0.114213917242679

C:\Users\Mateus\AppData\Local\Temp\ipykernel_22600\3895918696.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return np.where(old != 0, ((new - old) / old) * 100, np.nan)


,Metric,ONESTEP_MEAN,ONESTEP_STD,TWOSTEP_MEAN,TWOSTEP_STD,MEAN_DIFF_%,STD_DIFF_%,POINTWISE_MEAN_%,POINTWISE_STD_%
0,Time,0.444768,0.066737,0.702169,0.113987,57.873256,70.799789,59.415213,25.652496
1,Size,9.858209,1.159986,9.858209,1.159986,0.000000,0.0,0.000000,0.000000
2,Ranges_Sum,5.871908,1.129224,5.983612,0.924304,1.902341,-18.146964,4.222384,18.814433
3,Coverage,1.000000,0.000000,1.000000,0.000000,0.000000,nan,0.000000,0.000000


In [12]:
twostep_explanations[7]

[[0.0, 1.0],
 [0.1896103806539541, 0.7717762313996047],
 [0.0, 0.9999999999999999],
 [0.0, 0.28867229641878256],
 [0.5106140345076803, 0.7000881911023933],
 [0.7482442043475941, 1.0],
 [0.5056237684018499, 0.6226399349786771],
 [0.0, 0.39622641509433965],
 [0.0, 0.6088328075709779],
 [0.4138225255972696, 0.4539413213367091],
 [0.3778945856541137, 0.3821138211382114],
 [0.7728937728937728, 1.0],
 [0.36875891583452214, 1.0]]

In [13]:
np.savez(f'{result_path}/twostep_explanations{p_value}_test.npz', 
         twostep_explanations=twostep_explanations)